In [25]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 14.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatibl

# Given Amazon product reviews (text data), classify each review’s sentiment as positive, negative, or neutral based on its content. For example, a review saying “This phone is amazing!” should be classified as positive, while “Terrible quality, broke in a week” is negative.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Amazon_Reviews.csv to Amazon_Reviews (1).csv


In [2]:
import pandas as pd
# Try with error handling
data = pd.read_csv("Amazon_Reviews.csv", on_bad_lines="skip", encoding="utf-8", engine="python")
data.head()

,Reviewer Name,Profile Link,Country,Review Count,Review Date,Rating,Review Title,Review Text,Date of Experience
0,Eugene ath,/users/66e8185ff1598352d6b3701a,US,1 review,2024-09-16T13:44:26.000Z,Rated 1 out of 5 stars,A Store That Doesn't Want to Sell Anything,"I registered on the website, tried to order a ...",16-Sep-24
1,Daniel ohalloran,/users/5d75e460200c1f6a6373648c,GB,9 reviews,2024-09-16T18:26:46.000Z,Rated 1 out of 5 stars,Had multiple orders one turned up and…,Had multiple orders one turned up and driver h...,16-Sep-24
2,p fisher,/users/546cfcf1000064000197b88f,GB,90 reviews,2024-09-16T21:47:39.000Z,Rated 1 out of 5 stars,I informed these reprobates,I informed these reprobates that I WOULD NOT B...,16-Sep-24
3,Greg Dunn,/users/62c35cdbacc0ea0012ccaffa,AU,5 reviews,2024-09-17T07:15:49.000Z,Rated 1 out of 5 stars,Advertise one price then increase it on website,I have bought from Amazon before and no proble...,17-Sep-24
4,Sheila Hannah,/users/5ddbe429478d88251550610e,GB,8 reviews,2024-09-16T18:37:17.000Z,Rated 1 out of 5 stars,If I could give a lower rate I would,If I could give a lower rate I would! I cancel...,16-Sep-24


In [3]:
print(data.info(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21214 entries, 0 to 21213
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Reviewer Name       21214 non-null  object
 1   Profile Link        21163 non-null  object
 2   Country             21054 non-null  object
 3   Review Count        21055 non-null  object
 4   Review Date         21055 non-null  object
 5   Rating              21055 non-null  object
 6   Review Title        21055 non-null  object
 7   Review Text         21055 non-null  object
 8   Date of Experience  20947 non-null  object
dtypes: object(9)
memory usage: 1.5+ MB
None


Data frame, containing 21,214 entries across 9 columns, consists entirely of object data types and has missing values in several columns

In [4]:
review_count = len(data)
print(f"Total Review Count: {review_count}")

Total Review Count: 21214


In [5]:
data['Review Count'] = data['Review Count'].str.replace(' reviews', '', regex=False).str.replace(' review', '', regex=False).astype(float).fillna(0).astype(int)

In [6]:
print(data['Review Count'].head(10))

0     1
1     9
2    90
3     5
4     8
5     4
6    30
7     2
8     1
9     3
Name: Review Count, dtype: int64


In [7]:
data['Review Date'] = pd.to_datetime(data['Review Date'], errors='coerce')
data['Date of Experience'] = pd.to_datetime(data['Date of Experience'], errors='coerce')

# Now you can check how many values were successfully converted
print(f"Number of valid 'Review Date' entries: {data['Review Date'].notna().sum()}")
print(f"Number of valid 'Date of Experience' entries: {data['Date of Experience'].notna().sum()}")

Number of valid 'Review Date' entries: 21055
Number of valid 'Date of Experience' entries: 20947


/tmp/ipython-input-1536767063.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['Date of Experience'] = pd.to_datetime(data['Date of Experience'], errors='coerce')


In [8]:
print(data['Review Date'].head(10))
print(data['Date of Experience'].head(10))

0   2024-09-16 13:44:26+00:00
1   2024-09-16 18:26:46+00:00
2   2024-09-16 21:47:39+00:00
3   2024-09-17 07:15:49+00:00
4   2024-09-16 18:37:17+00:00
5   2024-09-17 10:07:38+00:00
6   2024-09-16 23:43:32+00:00
7   2024-09-15 18:07:45+00:00
8   2024-09-17 13:00:12+00:00
9   2024-09-16 21:04:07+00:00
Name: Review Date, dtype: datetime64[ns, UTC]
0   2024-09-16
1   2024-09-16
2   2024-09-16
3   2024-09-17
4   2024-09-16
5   2024-08-16
6   2024-09-16
7   2024-09-15
8   2024-09-17
9   2024-09-13
Name: Date of Experience, dtype: datetime64[ns]


In [9]:
# Suppose columns: 'reviewText' and 'sentiment' (adjust as per dataset)
data = data[['Review Text', 'Rating']]

print(data.shape)
print(data.head(5))

(21214, 2)
                                         Review Text                  Rating
0  I registered on the website, tried to order a ...  Rated 1 out of 5 stars
1  Had multiple orders one turned up and driver h...  Rated 1 out of 5 stars
2  I informed these reprobates that I WOULD NOT B...  Rated 1 out of 5 stars
3  I have bought from Amazon before and no proble...  Rated 1 out of 5 stars
4  If I could give a lower rate I would! I cancel...  Rated 1 out of 5 stars


In [10]:
import re

# Function to extract rating number from "Rated X out of 5 stars"
def extract_rating(text):
    if pd.isna(text):  # handle NaN
        return None
    match = re.search(r'(\d) out of 5', str(text))
    if match:
        return int(match.group(1))
    return None

# Apply extraction
data['Rating_num'] = data['Rating'].apply(extract_rating)

# Now create sentiment labels
def label_sentiment(rating):
    if pd.isna(rating):
        return None
    if rating <= 2:
        return "negative"
    elif rating == 3:
        return "neutral"
    else:
        return "positive"

data['Sentiment'] = data['Rating_num'].apply(label_sentiment)

# Check results
print(data[['Rating', 'Rating_num', 'Sentiment']].head(10))
print(data['Sentiment'].value_counts())

                   Rating  Rating_num Sentiment
0  Rated 1 out of 5 stars         1.0  negative
1  Rated 1 out of 5 stars         1.0  negative
2  Rated 1 out of 5 stars         1.0  negative
3  Rated 1 out of 5 stars         1.0  negative
4  Rated 1 out of 5 stars         1.0  negative
5  Rated 1 out of 5 stars         1.0  negative
6  Rated 1 out of 5 stars         1.0  negative
7  Rated 5 out of 5 stars         5.0  positive
8  Rated 1 out of 5 stars         1.0  negative
9  Rated 5 out of 5 stars         5.0  positive
Sentiment
negative    14350
positive     5820
neutral       885
Name: count, dtype: int64


In [11]:
print(data['Rating'].head(10))
print(data['Rating_num'].head(10))
print(data['Sentiment'].head(10))

0    Rated 1 out of 5 stars
1    Rated 1 out of 5 stars
2    Rated 1 out of 5 stars
3    Rated 1 out of 5 stars
4    Rated 1 out of 5 stars
5    Rated 1 out of 5 stars
6    Rated 1 out of 5 stars
7    Rated 5 out of 5 stars
8    Rated 1 out of 5 stars
9    Rated 5 out of 5 stars
Name: Rating, dtype: object
0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
6    1.0
7    5.0
8    1.0
9    5.0
Name: Rating_num, dtype: float64
0    negative
1    negative
2    negative
3    negative
4    negative
5    negative
6    negative
7    positive
8    negative
9    positive
Name: Sentiment, dtype: object


In [12]:
# Convert Rating_num to nullable integer
data['Rating_num'] = pd.to_numeric(data['Rating_num'], errors='coerce')
data['Rating_num'] = data['Rating_num'].astype('Int64')  # Nullable integer

# Convert Sentiment to categorical (more efficient)
data['Sentiment'] = data['Sentiment'].astype('category')

# Check the results
print(data.info())
print(data['Rating_num'].dtype)  # Should show Int64
print(data['Sentiment'].dtype)   # Should show category

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21214 entries, 0 to 21213
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Review Text  21055 non-null  object  
 1   Rating       21055 non-null  object  
 2   Rating_num   21055 non-null  Int64   
 3   Sentiment    21055 non-null  category
dtypes: Int64(1), category(1), object(2)
memory usage: 538.9+ KB
None
Int64
category


In [13]:
# Remove rows with missing ratings before conversion
data_clean = data.dropna(subset=['Rating_num', 'Sentiment'])

# Now convert to proper types
data_clean['Rating_num'] = data_clean['Rating_num'].astype('int64')
data_clean['Sentiment'] = data_clean['Sentiment'].astype('category')

print(data_clean.info())

<class 'pandas.core.frame.DataFrame'>
Index: 21055 entries, 0 to 21213
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Review Text  21055 non-null  object  
 1   Rating       21055 non-null  object  
 2   Rating_num   21055 non-null  int64   
 3   Sentiment    21055 non-null  category
dtypes: category(1), int64(1), object(2)
memory usage: 678.7+ KB
None


/tmp/ipython-input-2431889522.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['Rating_num'] = data_clean['Rating_num'].astype('int64')
/tmp/ipython-input-2431889522.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['Sentiment'] = data_clean['Sentiment'].astype('category')


In [14]:
# Check what's actually being extracted
print("Sample Rating values:")
print(data['Rating'].head(20))
print("\nCorresponding Rating_num values:")
print(data['Rating_num'].head(20))
print("\nUnique Rating formats:")
print(data['Rating'].value_counts().head(10))

Sample Rating values:
0     Rated 1 out of 5 stars
1     Rated 1 out of 5 stars
2     Rated 1 out of 5 stars
3     Rated 1 out of 5 stars
4     Rated 1 out of 5 stars
5     Rated 1 out of 5 stars
6     Rated 1 out of 5 stars
7     Rated 5 out of 5 stars
8     Rated 1 out of 5 stars
9     Rated 5 out of 5 stars
10    Rated 5 out of 5 stars
11    Rated 1 out of 5 stars
12    Rated 2 out of 5 stars
13    Rated 1 out of 5 stars
14    Rated 1 out of 5 stars
15    Rated 1 out of 5 stars
16    Rated 1 out of 5 stars
17    Rated 1 out of 5 stars
18    Rated 1 out of 5 stars
19    Rated 2 out of 5 stars
Name: Rating, dtype: object

Corresponding Rating_num values:
0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     5
8     1
9     5
10    5
11    1
12    2
13    1
14    1
15    1
16    1
17    1
18    1
19    2
Name: Rating_num, dtype: Int64

Unique Rating formats:
Rating
Rated 1 out of 5 stars    13123
Rated 5 out of 5 stars     4528
Rated 4 out of 5 stars     1292
Rated 2 out of 5 s

In [15]:
print(data['Rating'].unique()[:10])  # Show first 10 unique values

['Rated 1 out of 5 stars' 'Rated 5 out of 5 stars'
 'Rated 2 out of 5 stars' 'Rated 4 out of 5 stars'
 'Rated 3 out of 5 stars' nan]


In [16]:
print(f"None values in Rating_num: {data['Rating_num'].isna().sum()}")
print(f"None values in Sentiment: {data['Sentiment'].isna().sum()}")

None values in Rating_num: 159
None values in Sentiment: 159


In [17]:
# Test your regex on sample data
test_ratings = data['Rating'].dropna().head(10)
for rating in test_ratings:
    match = re.search(r'(\d) out of 5', str(rating))
    print(f"'{rating}' → {match.group(1) if match else 'No match'}")

'Rated 1 out of 5 stars' → 1
'Rated 1 out of 5 stars' → 1
'Rated 1 out of 5 stars' → 1
'Rated 1 out of 5 stars' → 1
'Rated 1 out of 5 stars' → 1
'Rated 1 out of 5 stars' → 1
'Rated 1 out of 5 stars' → 1
'Rated 5 out of 5 stars' → 5
'Rated 1 out of 5 stars' → 1
'Rated 5 out of 5 stars' → 5


In [18]:
def extract_rating_improved(text):
    if pd.isna(text):
        return None
    text = str(text).lower()

    # Try multiple patterns
    patterns = [
        r'(\d)(?:\.\d)?\s*out of 5',  # "4 out of 5" or "4.5 out of 5"
        r'rated\s+(\d)',              # "Rated 4"
        r'^(\d)(?:\.\d)?$',           # Just "4" or "4.5"
    ]

    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            return float(match.group(1))

    return None

In [19]:
# Count how many extractions failed
print("Extraction success rate:")
print(f"Total rows: {len(data)}")
print(f"Successful extractions: {data['Rating_num'].notna().sum()}")
print(f"Failed extractions: {data['Rating_num'].isna().sum()}")

Extraction success rate:
Total rows: 21214
Successful extractions: 21055
Failed extractions: 159


In [20]:
# 1. Improve extraction
data['Rating_num'] = data['Rating'].apply(extract_rating_improved)

# 2. Convert to proper types
data['Rating_num'] = pd.to_numeric(data['Rating_num'], errors='coerce').astype('Int64')
data['Sentiment'] = data['Sentiment'].astype('category')

# 3. Clean data (remove failed extractions)
data_clean = data.dropna(subset=['Rating_num', 'Sentiment']).copy()

# 4. Verify
print(data_clean.info())
print(f"Final dataset size: {len(data_clean)} rows")

<class 'pandas.core.frame.DataFrame'>
Index: 21055 entries, 0 to 21213
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Review Text  21055 non-null  object  
 1   Rating       21055 non-null  object  
 2   Rating_num   21055 non-null  Int64   
 3   Sentiment    21055 non-null  category
dtypes: Int64(1), category(1), object(2)
memory usage: 699.2+ KB
None
Final dataset size: 21055 rows


In [21]:
# Check the class distribution
print("Sentiment Distribution:")
print(data_clean['Sentiment'].value_counts())
print("\nPercentage Distribution:")
print(data_clean['Sentiment'].value_counts(normalize=True) * 100)

Sentiment Distribution:
Sentiment
negative    14350
positive     5820
neutral       885
Name: count, dtype: int64

Percentage Distribution:
Sentiment
negative    68.154833
positive    27.641890
neutral      4.203277
Name: proportion, dtype: float64


In [22]:
# Rating distribution
print("Rating Distribution:")
print(data_clean['Rating_num'].value_counts().sort_index())

# Sample some reviews by sentiment
print("\nSample Negative Reviews:")
print(data_clean[data_clean['Sentiment'] == 'negative']['Review Text'].head(3).values)

print("\nSample Positive Reviews:")
print(data_clean[data_clean['Sentiment'] == 'positive']['Review Text'].head(3).values)

Rating Distribution:
Rating_num
1    13123
2     1227
3      885
4     1292
5     4528
Name: count, dtype: Int64

Sample Negative Reviews:
["I registered on the website, tried to order a laptop, entered all the details, but instead of charging me and sending the product, they froze my account, demanding various verification documents. I sent them over. They said they would review them within 24 hours. In reality, it's been a week, and no one can help or give any (truthful) estimate of when it will be resolved; they just tell me to 'wait.' I've never seen such a horrible marketplace in my life. I hope those who came up with this can't buy food in a store, receiving a 'document review request' that takes forever to process."
 "Had multiple orders one turned up and driver had to phone as no door number on packaging, then waited all day for second package to get a message saying couldn't deliver as no number on packaging, 12 hours waiting in now don't even know when I'm getting delivery. T

In [23]:
import nltk
import re
from sklearn.model_selection import train_test_split

def clean_text(text):
    """Clean and preprocess review text"""
    if pd.isna(text):
        return ""

    # Convert to lowercase
    text = str(text).lower()

    # Remove URLs, mentions, hashtags
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

# Apply text cleaning
data_clean['Clean_Review'] = data_clean['Review Text'].apply(clean_text)

# Remove empty reviews
data_clean = data_clean[data_clean['Clean_Review'].str.len() > 10].copy()

print(f"After text cleaning: {len(data_clean)} rows")

After text cleaning: 21047 rows


In [24]:
from sklearn.model_selection import train_test_split

# Prepare features and target (this step was missing!)
X = data_clean['Clean_Review']  # Your cleaned text
y = data_clean['Sentiment']     # Your sentiment labels

# Create train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Keep same class distribution
)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")
print(f"\nOriginal training distribution:")
print(y_train.value_counts(normalize=True))

Training set: 16837 samples
Test set: 4210 samples

Original training distribution:
Sentiment
negative    0.681832
positive    0.276118
neutral     0.042050
Name: proportion, dtype: float64


In [25]:
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

print("🔄 Applying SMOTE to balance classes...")

# Vectorize text for SMOTE (it needs numerical features)
tfidf_temp = TfidfVectorizer(max_features=1000, stop_words='english')
X_tfidf_temp = tfidf_temp.fit_transform(X_train)

print(f"TF-IDF shape before SMOTE: {X_tfidf_temp.shape}")
print(f"Class distribution before SMOTE:")
print(y_train.value_counts())

# Apply SMOTE
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X_tfidf_temp, y_train)

print(f"\nTF-IDF shape after SMOTE: {X_balanced.shape}")
print("After SMOTE balancing:")
print(pd.Series(y_balanced).value_counts())
print("\nPercentage distribution:")
print(pd.Series(y_balanced).value_counts(normalize=True))

🔄 Applying SMOTE to balance classes...
TF-IDF shape before SMOTE: (16837, 1000)
Class distribution before SMOTE:
Sentiment
negative    11480
positive     4649
neutral       708
Name: count, dtype: int64

TF-IDF shape after SMOTE: (34440, 1000)
After SMOTE balancing:
Sentiment
negative    11480
neutral     11480
positive    11480
Name: count, dtype: int64

Percentage distribution:
Sentiment
negative    0.333333
neutral     0.333333
positive    0.333333
Name: proportion, dtype: float64


In [26]:
# Traditional (what you're comparing against)
from sklearn.feature_extraction.text import CountVectorizer
bow_model = CountVectorizer()  # One-hot encoding approach

# Modern approach (what you're implementing)
from gensim.models import Word2Vec
w2v_model = Word2Vec()  # Distributed word representations

In [27]:
# Train on your balanced data
sentences = [text.split() for text in X_train]  # Tokenize
w2v_model = Word2Vec(
    sentences=sentences,
    vector_size=100,    # Manning discusses dimensionality
    window=5,           # Context window (key Lecture 2 concept)
    sg=0,               # CBOW vs Skip-gram
    negative=5          # Negative sampling (Manning explains this)
)

# Test semantic relationships (Manning's favorite demo)
similar_words = w2v_model.wv.most_similar('good', topn=5)
print(f"Words similar to 'good': {similar_words}")

Words similar to 'good': [('great', 0.8649375438690186), ('convenient', 0.718320906162262), ('excellent', 0.6967635750770569), ('nice', 0.6855817437171936), ('bad', 0.685529887676239)]


In [35]:
similar_words = w2v_model.wv.most_similar('truth', topn=5)
print(f"Words similar to 'truth': {similar_words}")

Words similar to 'truth': [('word', 0.6978323459625244), ('infringement', 0.6464583277702332), ('jobs', 0.6449962258338928), ('script', 0.6355670690536499), ('america', 0.6323620676994324)]


In [56]:
# =============================
# Imports
# =============================
import numpy as np
import gradio as gr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# =============================
# Sentiment Analyzer Class
# =============================
class SentimentAnalyzer:
    def __init__(self):
        self.tfidf_vectorizer = None
        self.tfidf_model = None
        self.w2v_model = None
        self.w2v_classifier = None
        self.label_encoder = None

    def train(self, X_train, y_train):
        # Encode labels
        self.label_encoder = LabelEncoder()
        y_encoded = self.label_encoder.fit_transform(y_train)

        # TF-IDF
        self.tfidf_vectorizer = TfidfVectorizer(max_features=5000)
        X_tfidf = self.tfidf_vectorizer.fit_transform(X_train)
        self.tfidf_model = LogisticRegression(max_iter=500)
        self.tfidf_model.fit(X_tfidf, y_encoded)

        # Word2Vec
        tokenized = [x.split() for x in X_train]
        self.w2v_model = Word2Vec(sentences=tokenized, vector_size=100, window=5, min_count=1, workers=4)

        X_w2v = np.array([np.mean([self.w2v_model.wv[w] for w in words if w in self.w2v_model.wv] or
                                  [np.zeros(100)], axis=0) for words in tokenized])
        self.w2v_classifier = LogisticRegression(max_iter=500)
        self.w2v_classifier.fit(X_w2v, y_encoded)

    def predict_tfidf(self, text):
        vec = self.tfidf_vectorizer.transform([text])
        probs = self.tfidf_model.predict_proba(vec)[0]
        idx = np.argmax(probs)
        return self.label_encoder.classes_[idx], dict(zip(self.label_encoder.classes_, probs))

    def predict_w2v(self, text):
        words = text.split()
        vec = np.mean([self.w2v_model.wv[w] for w in words if w in self.w2v_model.wv] or
                      [np.zeros(100)], axis=0).reshape(1, -1)
        probs = self.w2v_classifier.predict_proba(vec)[0]
        idx = np.argmax(probs)
        return self.label_encoder.classes_[idx], dict(zip(self.label_encoder.classes_, probs))

# =============================
# Neural Network (Lecture 3)
# =============================
class SentimentMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SentimentMLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# Train simple NN on same data
def train_neural(X_train, y_train, w2v_model, label_encoder, epochs=30):
    # Convert training sentences to Word2Vec average vectors
    X_train_vectors = np.array([
        np.mean([w2v_model.wv[w] for w in x.split() if w in w2v_model.wv] or
                [np.zeros(100)], axis=0) for x in X_train
    ])
    y_encoded = label_encoder.transform(y_train)

    # Convert to tensors
    X_tensor = torch.tensor(X_train_vectors, dtype=torch.float32)
    y_tensor = torch.tensor(y_encoded, dtype=torch.long)

    # Initialize model with correct input size
    #print("\n🧠 Step 4: Initializing Neural Network...")
    model = SentimentMLP(
        input_dim=X_train_vectors.shape[1],     # ✅ FIXED
        hidden_dim=128,
        output_dim=len(label_encoder.classes_)
    )

    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    for _ in range(epochs):
        optimizer.zero_grad()
        out = model(X_tensor)
        loss = F.cross_entropy(out, y_tensor)
        loss.backward()
        optimizer.step()

    return model

def predict_neural(text, model, w2v_model, label_encoder):
    words = text.split()
    vec = np.mean([w2v_model.wv[w] for w in words if w in w2v_model.wv] or
                  [np.zeros(100)], axis=0)
    vec_tensor = torch.tensor(vec, dtype=torch.float32).unsqueeze(0)
    probs = F.softmax(model(vec_tensor), dim=1).detach().numpy()[0]
    idx = np.argmax(probs)
    return label_encoder.classes_[idx], dict(zip(label_encoder.classes_, probs))

# =============================
# Training Data
# =============================
X_train = [
    "I love this phone, it is fantastic!",
    "Terrible product, complete waste of money.",
    "It’s okay, not too bad but not great either.",
    "Amazing quality, I am very happy with it.",
    "Worst purchase ever, I hate it."
]
y_train = ["positive", "negative", "neutral", "positive", "negative"]

# =============================
# Train Analyzer (TF-IDF + W2V)
# =============================
analyzer = SentimentAnalyzer()
analyzer.train(X_train, y_train)

# Save global Word2Vec and label encoder for NN
w2v_model = analyzer.w2v_model
label_encoder = analyzer.label_encoder

# Train NN
# Train NN
neural_model = train_neural(X_train, y_train, w2v_model, label_encoder)

# =============================
# Gradio Interface
# =============================
def analyze(text):
    tfidf_pred, tfidf_probs = analyzer.predict_tfidf(text)
    w2v_pred, w2v_probs = analyzer.predict_w2v(text)
    nn_pred, nn_probs = predict_neural(text, neural_model, w2v_model, label_encoder)

    return (
        f"TF-IDF Prediction: {tfidf_pred}\n\nProbabilities: {tfidf_probs}",
        f"Word2Vec Prediction: {w2v_pred}\n\nProbabilities: {w2v_probs}",
        f"Neural Network Prediction: {nn_pred}\n\nProbabilities: {nn_probs}"
    )

demo = gr.Interface(
    fn=analyze,
    inputs=gr.Textbox(lines=2, placeholder="Enter a review..."),
    outputs=[
        gr.Textbox(label="TF-IDF Result"),
        gr.Textbox(label="Word2Vec Result"),
        gr.Textbox(label="Neural Network Result")
    ],
    title="📊 Sentiment Analyzer (TF-IDF vs Word2Vec vs Neural Net)"
)

In [58]:
# Train and initialize
#neural_model, vectorizer, label_encoder = run_cs224n_neural_sentiment()

# Now test examples
#test_neural_model(neural_model, vectorizer, label_encoder,
                 #"This product is absolutely amazing! Love it!")

#test_neural_model(neural_model, vectorizer, label_encoder,
                # "Terrible quality, complete waste of money!")

#test_neural_model(neural_model, vectorizer, label_encoder,
                 #"The product is okay, nothing special.")

#test_neural_model(neural_model, vectorizer, label_encoder,
                 #"Not bad at all, actually quite good!")

In [70]:
import gradio as gr
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import pickle
import re

class SentimentAnalyzer:
    def __init__(self):
        self.tfidf_model = None
        self.tfidf_vectorizer = None
        self.w2v_model = None
        self.w2v_classifier = None
        self.is_trained = False  # Track if models are trained

    def preprocess_text(self, text):
        """Clean and preprocess text"""
        # Convert to lowercase
        text = text.lower()
        # Remove special characters and digits
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        # Remove extra whitespace
        text = ' '.join(text.split())
        return text

    def create_sample_data(self):
        """Create sample training data since no actual data is provided"""
        print("Creating sample training data...")

        # Sample positive reviews
        positive_reviews = [
            "This product is absolutely amazing! I love it.",
            "Excellent quality and great value for money.",
            "Highly recommended. Works perfectly as described.",
            "Outstanding performance and durability.",
            "Fantastic product with amazing features.",
            "Very satisfied with this purchase.",
            "Top notch quality and excellent customer service.",
            "Impressive product that exceeded my expectations.",
            "Wonderful experience using this product.",
            "Great value and excellent performance."
        ]

        # Sample negative reviews
        negative_reviews = [
            "Terrible product. Complete waste of money.",
            "Poor quality and does not work as advertised.",
            "Very disappointed with this purchase.",
            "Stopped working after just a few days.",
            "Low quality materials and bad craftsmanship.",
            "Not worth the price at all.",
            "Defective product, would not recommend.",
            "Extremely dissatisfied with this item.",
            "Cheaply made and broke quickly.",
            "Worst purchase I've made in a long time."
        ]

        # Sample neutral reviews
        neutral_reviews = [
            "It's okay, nothing special but gets the job done.",
            "Average product with average performance.",
            "Not great but not terrible either.",
            "Mediocre quality for the price.",
            "Does what it's supposed to but nothing more.",
            "Acceptable but I expected better.",
            "Fair product for the price point.",
            "Neither impressed nor disappointed.",
            "It works but there are better options.",
            "Adequate but not exceptional in any way."
        ]

        # Combine all reviews and create labels
        reviews = positive_reviews + negative_reviews + neutral_reviews
        labels = (["positive"] * len(positive_reviews) +
                 ["negative"] * len(negative_reviews) +
                 ["neutral"] * len(neutral_reviews))

        return reviews, labels

    def train_models(self):
        """Train both TF-IDF and Word2Vec models with sample data"""
        print("Training models with sample data...")

        # Create sample data
        X_train, y_train = self.create_sample_data()

        # TF-IDF Model
        self.tfidf_vectorizer = TfidfVectorizer(
            max_features=5000,
            ngram_range=(1, 2),
            min_df=1,  # Changed to 1 for small sample data
            max_df=0.95,  # Adjusted for small sample data
            stop_words='english'
        )

        X_train_tfidf = self.tfidf_vectorizer.fit_transform(X_train)
        self.tfidf_model = LogisticRegression(random_state=42, max_iter=1000)
        self.tfidf_model.fit(X_train_tfidf, y_train)

        print("Training Word2Vec model...")
        # Word2Vec Model
        train_tokens = [simple_preprocess(text, deacc=True) for text in X_train]
        self.w2v_model = Word2Vec(
            sentences=train_tokens,
            vector_size=100,
            window=5,
            min_count=1,  # Changed to 1 for small sample data
            workers=4,
            sg=0,  # CBOW
            epochs=10
        )

        # Convert to vectors
        X_train_w2v = np.array([self._review_to_vector(tokens) for tokens in train_tokens])
        self.w2v_classifier = LogisticRegression(random_state=42, max_iter=1000)
        self.w2v_classifier.fit(X_train_w2v, y_train)

        self.is_trained = True
        print("Models trained successfully!")

    def _review_to_vector(self, tokens):
        """Convert tokenized review to Word2Vec vector"""
        if not self.w2v_model:
            return np.zeros(100)  # Return zero vector if model not trained

        word_vectors = [self.w2v_model.wv[word] for word in tokens
                       if word in self.w2v_model.wv.key_to_index]

        if len(word_vectors) == 0:
            return np.zeros(self.w2v_model.wv.vector_size)
        else:
            return np.mean(word_vectors, axis=0)

    def predict_tfidf(self, text):
        """Predict using TF-IDF model"""
        if not self.is_trained:
            return "Model not trained", {"positive": 0.33, "negative": 0.33, "neutral": 0.33}

        processed_text = self.preprocess_text(text)
        text_vector = self.tfidf_vectorizer.transform([processed_text])

        # Get prediction and probabilities
        prediction = self.tfidf_model.predict(text_vector)[0]
        probabilities = self.tfidf_model.predict_proba(text_vector)[0]

        # Get class names
        classes = self.tfidf_model.classes_
        prob_dict = {classes[i]: float(probabilities[i]) for i in range(len(classes))}

        return prediction, prob_dict

    def predict_word2vec(self, text):
        """Predict using Word2Vec model"""
        if not self.is_trained:
            return "Model not trained", {"positive": 0.33, "negative": 0.33, "neutral": 0.33}

        processed_text = self.preprocess_text(text)
        tokens = simple_preprocess(processed_text, deacc=True)
        text_vector = self._review_to_vector(tokens).reshape(1, -1)

        # Get prediction and probabilities
        prediction = self.w2v_classifier.predict(text_vector)[0]
        probabilities = self.w2v_classifier.predict_proba(text_vector)[0]

        # Get class names
        classes = self.w2v_classifier.classes_
        prob_dict = {classes[i]: float(probabilities[i]) for i in range(len(classes))}

        return prediction, prob_dict

    def get_similar_words(self, word):
        """Get similar words using Word2Vec"""
        if not self.is_trained:
            return "Model not trained yet. Please train the model first."

        try:
            if word.lower() in self.w2v_model.wv.key_to_index:
                similar_words = self.w2v_model.wv.most_similar(word.lower(), topn=5)
                return similar_words
            else:
                return f"Word '{word}' not found in vocabulary"
        except:
            return "Error finding similar words"

# Initialize and train the analyzer
analyzer = SentimentAnalyzer()
analyzer.train_models()  # Train with sample data

def analyze_sentiment(text, model_choice):
    """Main function for Gradio interface"""
    if not text.strip():
        return "Please enter some text to analyze.", {}, ""

    try:
        if model_choice == "TF-IDF":
            prediction, probabilities = analyzer.predict_tfidf(text)
            model_info = "🔤 TF-IDF: Frequency-based word representation"
        else:  # Word2Vec
            prediction, probabilities = analyzer.predict_word2vec(text)
            model_info = "🧠 Word2Vec: Semantic word embeddings (CS224N!)"

        return f"**Prediction: {prediction}**", probabilities, model_info

    except Exception as e:
        return f"Error: {str(e)}", {}, ""

def find_similar_words(word):
    """Find similar words using Word2Vec"""
    try:
        similar = analyzer.get_similar_words(word)
        if isinstance(similar, str):
            return similar
        else:
            result = f"Words similar to '{word}':\n"
            for word_sim, score in similar:
                result += f"• {word_sim} (similarity: {score:.3f})\n"
            return result
    except Exception as e:
        return f"Error: {str(e)}"

# Create Gradio Interface
with gr.Blocks(title="Sentiment Analysis: TF-IDF vs Word2Vec", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🎭 Sentiment Analysis Comparison")
    gr.Markdown("### TF-IDF vs Word2Vec Models (CS224N Implementation)")

    with gr.Tab("🔍 Sentiment Analysis"):
        with gr.Row():
            with gr.Column(scale=2):
                text_input = gr.Textbox(
                    label="Enter text to analyze:",
                    placeholder="This product is amazing! I love it.",
                    lines=3
                )
                model_choice = gr.Radio(
                    choices=["TF-IDF", "Word2Vec"],
                    label="Choose Model:",
                    value="TF-IDF"
                )
                analyze_btn = gr.Button("🔍 Analyze Sentiment", variant="primary")

            with gr.Column(scale=2):
                prediction_output = gr.Markdown(label="Prediction")
                probabilities_output = gr.Label(label="Confidence Scores")
                model_info = gr.Markdown(label="Model Info")

        # Example texts
        gr.Markdown("### 📝 Try These Examples:")
        gr.Examples([
            "This product is absolutely terrible! Waste of money.",
            "The item is okay, nothing special but works fine.",
            "Amazing quality! Exceeded my expectations completely!",
            "The service was poor and the product arrived damaged."
        ], inputs=text_input)

    with gr.Tab("🔤 Word2Vec Similar Words"):
        gr.Markdown("## Find Similar Words using Word2Vec")
        word_input = gr.Textbox(
            label="Enter a word to find similar words:",
            placeholder="amazing",
            lines=1
        )
        similar_btn = gr.Button("Find Similar Words", variant="primary")
        similar_output = gr.Textbox(label="Similar Words", lines=6)

        similar_btn.click(
            fn=find_similar_words,
            inputs=word_input,
            outputs=similar_output
        )

    # Connect the analyze button
    analyze_btn.click(
        fn=analyze_sentiment,
        inputs=[text_input, model_choice],
        outputs=[prediction_output, probabilities_output, model_info]
    )

Training models with sample data...
Creating sample training data...
Training Word2Vec model...
Models trained successfully!


In [71]:
import sys

if "google.colab" in sys.modules:
    demo.launch(share=True, inline=False)  # For Colab
else:
    demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4156cdf5cb81a73aaf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
